# Logistic Regression

- H(x) = 1 / (1 + e^( (-W)^T * X ) )
- cost(W) = -1 / m ∑ ( ylog(H(x)) + (1 - y)log(1 - H(x)) )

logistic regression은 dynamic classification 문제이다. 
- 확률 기반으로 여러 데이터를 0, 1로 분류한다. 
- 즉, P(y = 1 | X) 같은 확률을 예측하고 임계값을 기준으로 0, 1을 분류하는 방식이다. 
- dynamic classification이란, 정확히 0/1이 아닌 값들에 대한 분류이다. X가 바뀔 때 마다 동적으로 결과가 바뀔 수 있으며, 가중치에 따라 입력을 받아 분류함

m개의 샘플 데이터가 d의 차원으로 존재할 때 m개의 0, 1로 만들어준다. 

경사하강법을 통한 가중치 변화값
```
w := w - a * d(cost(w)) / d(w) (a = learning rate)
```
가중치 - 학습률 * cost함수 미분값

In [82]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [83]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

|x_data| = (6, 2)
|y_data| = (6, 1)

In [84]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [85]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computing Hypothesis

위의 H(x)를 코드로 구현

In [86]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [87]:
W = torch.zeros((2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad=True)

In [88]:
hypothesis_by_manual = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [89]:
print(hypothesis_by_manual)
print(hypothesis_by_manual.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


사실 이렇게 할 필요 없이 torch.sigmoid를 이용하면 된다. 
sigmoid함수란 sig(x) = 1 / (1 + e ^ (-x))로, x대신에 우리가 윈하는 값을 넣어주면 된다. 

결국 아래의 식이 성립하게 된다. 

```
1 / (1 + torch.exp(-(x_train.matmul(W) + b))) == torch.sigmoid(x_train.matmul(W) + b)
```

In [90]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


## Cost Function

위에 작성한 cost(W) = -1 / m ∑ ( ylog(H(x)) + (1 - y)log(1 - H(x)) ) 라는 식을 직접 구현하는 과정이다. 
여기에서 H(x)란, p(x=1; w)로, w가 주어졌을 때 x가 1일 확률이다. 

In [91]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [92]:
losses = -(y_train * torch.log(hypothesis) + (1-y_train) * torch.log(1 - hypothesis))
print(losses)

cost = losses.mean()
print(cost)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)
tensor(0.6931, grad_fn=<MeanBackward0>)


이렇게 하지 말고 torch.nn.functional에 존재하는 binary_cross_entrophy를 이용하면 편하다. 
이걸 줄여서 bce라고 하기도 한다. 

In [93]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

## Whole Training

In [94]:
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000

for epoch in range(nb_epochs+1):
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {: .6}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost:  0.693147
Epoch  100/1000 Cost:  0.134722
Epoch  200/1000 Cost:  0.0806431


Epoch  300/1000 Cost:  0.0579
Epoch  400/1000 Cost:  0.0452997
Epoch  500/1000 Cost:  0.037261
Epoch  600/1000 Cost:  0.0316725
Epoch  700/1000 Cost:  0.0275559
Epoch  800/1000 Cost:  0.0243943
Epoch  900/1000 Cost:  0.0218883
Epoch 1000/1000 Cost:  0.0198522


## Evaluation

In [95]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5].type(torch.uint8))

tensor([[0],
        [0],
        [0],
        [1],
        [1]], dtype=torch.uint8)


In [96]:
print(prediction[:5])
print(y_train[:5].type(torch.uint8))

correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0],
        [0],
        [0],
        [1],
        [1]], dtype=torch.uint8)
tensor([[True],
        [True],
        [True],
        [True],
        [True]])


이렇게 사용하기 보단, 클래스를 이용해 처리하는게 낫다. 

In [97]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1) 
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [98]:
model = BinaryClassifier()

위의 클래스를 보았을 때, Linear를 이용해 W, b 파라미터를 추가한다. 
이를 통해 로지스틱스 회귀에서, 샘플 데이터의 개수는 모르지만 2차원이라는 것을 알 수 있다. 

즉, W = (2, 1), b = (1)이라는 것을 알 수 있다. 


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100

for epoch in range(nb_epochs+1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        
        print('Epoch {:4d}/{} Cost: {:6f} Accuracy {:2.2f}%'.format(epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0/100 Cost: 0.539713 Accuracy 83.33
Epoch   10/100 Cost: 0.614853 Accuracy 66.67
Epoch   20/100 Cost: 0.441875 Accuracy 66.67
Epoch   30/100 Cost: 0.373145 Accuracy 83.33
Epoch   40/100 Cost: 0.316358 Accuracy 83.33
Epoch   50/100 Cost: 0.266094 Accuracy 83.33


Epoch   60/100 Cost: 0.220498 Accuracy 100.00
Epoch   70/100 Cost: 0.182095 Accuracy 100.00
Epoch   80/100 Cost: 0.157299 Accuracy 100.00
Epoch   90/100 Cost: 0.144091 Accuracy 100.00
Epoch  100/100 Cost: 0.134272 Accuracy 100.00
